# Special Methods on 'Object'
- methods of the form ```__...__```(dunder - double underscore) play special roles in implementing classes
- the first argument of almost all dunder methods is 'self'
- the class author defines these methods, but they are not normally called directly
    - some of the methods are invoked by operators
        - you can define what '+', '*', etc, means for an object you define
        - C++ allows this, Java doesn't
    - other methods are invoked by well known 'syntactic sugar' functions like 'len' and 'next'

# Basic
- ```__init__```  - called at object creation time. used to initialize object state
- ```__len__```  - 'len' function will call this method
- ```__bool__```  - 'bool' function will call this method
- ```__str__, __repr__``` - controls how object prints
- ```__contains__``` - used by 'in' operator
- ```__call__``` - call an object like a function call

In [1]:
class C:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        # 'friendly' string representation of object
        return('str method: ' + self.name)
    
    def __repr__(self):
        # 'detailed' string representation of object
        return('repr method: ' + self.name)
    
    def __len__(self):
        return len(self.name)
    
    def __bool__(self):
        # long name => true
        # short name => false
        # odd predicate
        return len(self.name) > 5
    
    def __call__(self, arg):
        # call an object like a function
        return arg + 10
    
    def __contains__(self, arg):
        # 'in' operator
        return arg in self.name

In [2]:
c = C('jack')

# top level uses __repr__
c

repr method: jack

In [3]:
# print uses __str__

print(c)

str method: jack


In [4]:
# note - 'in' operator will 
# search for substrings

'foo' in 'bazfoobar'

True

In [5]:
# calling __contains__

'ack' in c, 'jill' in c, c.__contains__('ack'), c.__contains__('jill')

(True, False, True, False)

In [6]:
# calling __bool__

[bool(C('jack')), bool(C('jackson'))]

[False, True]

In [7]:
# calling __len__

len(C('jackson'))

7

In [8]:
# call an object like a function - used by decorators
# calling __call__

c = C('afd')
c(34)

44

In [9]:
# 'repr' and 'str' are just methods, so an object can print 
# differently at any time, depending on whatever
# very common to display some kind of status summary for the object

import random

class C2:
    def __repr__(self):
        return(random.choice(['ready', 'waiting', 'running', 'finished']))
c2 = C2()
for j in range(10):
    print(c2)

waiting
running
ready
running
running
ready
ready
finished
waiting
waiting


# Iteration
- ```__iter__``` - return an 'iterable' for this object
    - 'iter' function calls this method
- ```__next__``` - call on an iterable to get the next element in the sequence. raises 'StopIteration' error when sequence is exhausted
    - 'next' function calls this method

In [10]:
# using the actual methods...

x = [32,4]
it = x.__iter__()
print(it)
print(it.__next__())
print(it.__next__())
it.__next__()


32
4


StopIteration: 

In [11]:
# using the top level functions - 
# exact same thing, but nicer looking
# "syntactic sugar"

x = [32,4]
it = iter(x)
print(it)
print(next(it))
print(next(it))
next(it)


32
4


StopIteration: 

# Comparision
- ```__lt__, __gt__, __le__, __ge__, __eq__, __ne__``` 
- tedious - many methods to define
- easier way is to use functools.total_ordering - only need to define two
- called by operators like '==' and '<'

In [12]:
from functools import total_ordering

# total_ordering is a 'decorator'
# it will 'write' the other four 
# predicates 'by magic'

@total_ordering
class Student:
    def __init__(self, first, last):
        # don't let case confuse sort
        self.firstname = first.lower()
        self.lastname = last.lower()
        
    def __eq__(self, other):
        return ((self.lastname, self.firstname) ==
                (other.lastname, other.firstname))
    
    def __lt__(self, other):
        return ((self.lastname, self.firstname) <
                (other.lastname, other.firstname))

s1 = Student('joe', 'college')
s2 = Student('jack', 'junior')

In [13]:
s1 is s2

False

In [14]:
s1 == s1

True

In [15]:
s1 == s2

False

In [16]:
s1 < s2

True

In [17]:
s1 > s2

False

# Hashing
- ```__hash__``` - should only be defined for immutable objects. the hash of a mutable object could change, making it a bogus key
- can turn it off this way

```
class foo:
    __hash__ = None
...
```

In [18]:
# calls __hash__

s = 'asdfadsfdsf'
hash(s)

6286453890542210990

In [19]:
# __hash__ method for list and dict is None, so they can'be dick keys

print({}.__hash__), print([].__hash__)

None
None


(None, None)

# collection element access and slices 
- ```__getitem__``` - get element or slice
- ```__setitem__``` - set element or slice

In [20]:
    def __getitem__(self, index):
        # do different things depending on type of 'index'
        if isinstance(index, int):
            # if asked for a single term, p[n], index will
            # be an int
            pass
        if isinstance(index, slice):
            # if asked for a slice, p[n:m], index will be
            # a 'slice' object
            pass
        pass

# Managed attributes
- sometimes you want to run code when a object attribute is accessed or set
- one way to do this is with decorators
- advanced technique

In [21]:
import random

# inherit from object
class RandomService:
    def __init__(self):
        self.counter = 0 
        
    @property
    #  this runs on a "get"
    # inc counter, rtn random
    def ran(self):
        self.counter += 1
        return(random.random())
    
    @ran.setter
    # this runs on a "set"
    # set seed, reset counter
    def ran(self, val):
        self.counter = 0
        if not isinstance(val, float):
            raise TypeError('Expected a float')
        random.seed(val)
    
rs = RandomService()

In [22]:
[rs.ran for j in range(5)]

[0.020673168648453366,
 0.23989751101707146,
 0.1739770930976693,
 0.41226341281046286,
 0.915222333700243]

In [23]:
# counter recorded 5 invocations

rs.counter

5

In [24]:
# wants a float
rs.ran = 'asfd'

TypeError: Expected a float

In [25]:
# set seed, reset counter

rs.ran = 1.0
rs.counter

0

In [26]:
[rs.ran for j in range(5)]

[0.13436424411240122,
 0.8474337369372327,
 0.763774618976614,
 0.2550690257394217,
 0.49543508709194095]

In [27]:
# reset seed, same random numbers

rs.ran = 1.0
[rs.ran for j in range(5)]

[0.13436424411240122,
 0.8474337369372327,
 0.763774618976614,
 0.2550690257394217,
 0.49543508709194095]